In [10]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import PIL

In [18]:
def read_write_image(img_name):
    img = Image.open(img_name)
    ext = input('Enter extension: ')
    return img.save(img.filename.split('.')[0] + '.' + ext)

In [ ]:
def show_image()

# B1: Tìm k centroids
## Vấn đề: Random không lấy lại màu cũ
# B2: Tính khoảng cách từ k centroids đến các điểm dữ liệu
## Sự thay đổi của centroid là không đáng kể hoặc không nhiều hoặc max_iter

### Màu cuối cùng tìm được không nhất thiết phải có trong ảnh
### Mask numpy